# Задание 1. Посчитайте количество женский имен в файле "names.txt".

# предварительная обработка текста:
выполнена в виду малого объема текста и неоднородности данных в текстовом редактореNotepad++ :
1) удалить все символы, кроме букв русского языка, запятых и пробелов, удалить ё:
    sub(r"[^А-Яа-яё .,\t\r\n]+"," ")
    sub(r"[ё]","е")
2) все пробельные символы заменить на одиночный пробел:
    sub(r"[\x01-\x09 ]+","")
    sub(r"[ ]+^","")
    sub(r"^[ ]+","")
3) удалить 1-е три строки
4) удалит любой текст после 2-ой запятой или точки
    sub(r"[^А-Яа-яё .,\t\r\n]+"," ")
5) очистить начало и конец от пробелов
6) удалить слова не с заглавной буквы
6) удалить пустые строки
7) отсортировать
8) удалить дубликаты (?) 
9) сохранить файл 

In [2]:
from pymorphy2 import MorphAnalyzer
#from pymystem3 import Mystem
import os, sys, time
print('Ok: ',time.asctime() )

Ok:  Wed Aug 22 05:57:10 2018


In [3]:
%%time
import pandas as pd
#path = os.getcwd()
#print(path)
print(os.path.exists('data/names_R.txt'), os.path.abspath('data/names_R.txt'))

df = pd.read_csv('data/names_R.txt',sep=',', header=0 ,encoding='utf-8',skip_blank_lines=True,names=['COL1']) # , usecols = ['text', 'source'])

print('Ok: ',time.asctime() )

True C:\Users\user-bsf\Tests\nlp-course-hse-summer-intensive\hw1\data\names_R.txt
Ok:  Wed Aug 22 05:57:12 2018
Wall time: 1.19 s


In [4]:
#удалить стоп слова 1
from nltk.corpus import stopwords
#print(stopwords.words('russian'))
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

#m = Mystem()
#def lemmatize(text, mystem=m):
#    try:
#        return "".join(m.lemmatize(text)).strip()  
#    except:
#        return " "

print('Ok: ',time.asctime() )

Ok:  Wed Aug 22 05:57:13 2018


In [5]:
%%time
#удалить стоп слова 2
df.COL1 = df.COL1.apply(remove_stopwords) 
#df.COL1 = df.COL1.apply(lemmatize)

m = MorphAnalyzer()
df.COL1=[m.parse(word)[0].normal_form for word in df.COL1]

f_names = []

for index, row in df.iterrows():
    word1=row['COL1']
    res=m.parse(word1)[0]
    res_tag = str(res.tag).split(",")
    if (res.score > 0.6) and (res_tag == ['NOUN', 'anim', 'femn', 'Name sing', 'nomn']) :
        #print(f'{res.normal_form}\t{res_tag}\t{res.score}')
        f_names.append(res.normal_form)
    #if index>=60:
        #print(index)
        #break

print(f'Итого, женских имен всего : {len(f_names)}.')
#print(f_names)
print('Ok: ',time.asctime() )

Итого, женских имен всего : 43.
Ok:  Wed Aug 22 05:57:14 2018
Wall time: 797 ms


In [6]:
%%time
from collections import Counter
n_types = []
n_tokens = []
tokens = []
cnt = Counter()
for f_name in f_names:
    #print(f_name)
    cnt[f_name] += 1
#for i in cnt.most_common():
#    print(i)
f_names2 = list(cnt)
#print(f_names2)
print(f'     , из них уникальных : {len(f_names2)}.')
for i in cnt.most_common():
    print(i)


     , из них уникальных : 20.
('марья', 15)
('анна', 4)
('наталья', 4)
('дуняша', 2)
('наташа', 2)
('репнина', 2)
('аграфена', 1)
('амалия', 1)
('аниска', 1)
('анисья', 1)
('елена', 1)
('елизавета', 1)
('жозефа', 1)
('катерина', 1)
('мария', 1)
('матрена', 1)
('настасья', 1)
('ольга', 1)
('пелагея', 1)
('прасковья', 1)
Wall time: 9.99 ms


# 'репнина' - скорее всего, неженское имя

Итого: 19 уникальных женских имен, 43 имени упомянуто. Вероятно, что часть из них указана как "имя" в отношении родства, но это, наверное, но такой анализ, не являлся часть задачи.